In [720]:
#ignore the warnings
options(warn=-1)

In [721]:
# Set environment params
Sys.setenv(LANG='en')  # English

# Import libraries
library(data.table)  # Data manipulate
library(mlr)         # ML toolkit
library(caret)       # ML toolkit
library(ggplot2)     # Visualization
library(pROC)        # AUC, ROC
library(e1071)       # Evaluation
library(gridExtra)   # Visualization
library(kknn)        # kNN model
library(nnet)        # One-vs-All Logistic Regression
library(dummies)     # Data processing

# Import data
library(ISLR)      # Data from the course book
library(MASS)      # Boston housing dataset
library(datasets)  # US crime dataset

# Resize plot
library(repr)  # String and binary representations
#options(repr.plot.width=4, repr.plot.height=4)

R.version.string

[1] "R version 4.0.5 (2021-03-31)"

In [722]:
#read in the datset
data = read.csv(".//Dataset 1_Bank Marketing//bank_mkt_train.csv")

In [723]:
summary(data) #get summary of the variables

   client_id          age            job              marital         
 Min.   :    2   Min.   :17.00   Length:20000       Length:20000      
 1st Qu.:10312   1st Qu.:32.00   Class :character   Class :character  
 Median :20762   Median :38.00   Mode  :character   Mode  :character  
 Mean   :20683   Mean   :40.05                                        
 3rd Qu.:30993   3rd Qu.:47.00                                        
 Max.   :41188   Max.   :98.00                                        
                 NA's   :202                                          
  education           default            housing              loan          
 Length:20000       Length:20000       Length:20000       Length:20000      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                               

In [724]:
# Train test separate with stratified at 70% for the train
set.seed(123)
train_idx <- createDataPartition(data$subscribe, p=.7, list=FALSE)
train <- data[train_idx, ]
test <- data[-train_idx, ]

In [725]:
print(dim(train))
print(dim(test))

[1] 14000    21
[1] 6000   21


## Treating NA values in the train set

In [726]:
colSums(is.na(train)) #get NAs per column

client_id            age            job        marital      education 
             0            143            114            147            120 
       default        housing           loan        contact          month 
           160            133            150            152            147 
   day_of_week       campaign          pdays       previous       poutcome 
           131            145            136            141            124 
  emp.var.rate cons.price.idx  cons.conf.idx      euribor3m    nr.employed 
           116            132            151            141            128 
     subscribe 
             0

In [727]:
#replacing NA with mean value of the column for some variables in a loop
cols = c("emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed")
for (i in cols){
train[is.na(train[,i]), i] <- mean(train[,i], na.rm = TRUE)
}

In [728]:
#replacing pdays missing values with 999 - no contact as most of the values are 999
train$pdays[is.na(train$pdays)] = 999
#replacing with 0 as most the values are 0
train$previous[is.na(train$previous)] = 0 

In [729]:
#dropping NA observations in campaign and age
train = train[complete.cases(train[ ,"campaign"]),]
train = train[complete.cases(train[ ,"age"]),]

In [730]:
colSums(is.na(train))

client_id            age            job        marital      education 
             0              0            111            143            118 
       default        housing           loan        contact          month 
           156            129            148            148            144 
   day_of_week       campaign          pdays       previous       poutcome 
           128              0              0              0            124 
  emp.var.rate cons.price.idx  cons.conf.idx      euribor3m    nr.employed 
             0              0              0              0              0 
     subscribe 
             0

In [732]:
# Get list of categorical variables in the train set
var_list <- names(train[, 1:(ncol(train)-1)])
cat_list <- var_list[sapply(train[, var_list], class) == 'character']
cat_list

[1] "job"         "marital"     "education"   "default"     "housing"    
 [6] "loan"        "contact"     "month"       "day_of_week" "poutcome"

In [733]:
# Convert categorical variables to dummy variables
dummy_list <- list()
for (v in cat_list) {
    # Create dummy variables
    tmp <- dummy(v, data=train, sep="_", drop=TRUE)
    # Drop the 1st column
    tmp <- tmp[, 2:ncol(tmp), drop=FALSE]
    # Store the results
    dummy_list[[length(dummy_list)+1]] <- tmp
}

# Combine the dummy variables
dummy_df <- do.call(cbind, dummy_list)

# Add the dummy variables to the data frame
train <- cbind(train, dummy_df)

# Drop the original variable
train <- train[, !(names(train) %in% cat_list)]

In [734]:
dim(train)

[1] 13713    64

In [735]:
colSums(is.na(train))

client_id                           age 
                            0                             0 
                     campaign                         pdays 
                            0                             0 
                     previous                  emp.var.rate 
                            0                             0 
               cons.price.idx                 cons.conf.idx 
                            0                             0 
                    euribor3m                   nr.employed 
                            0                             0 
                    subscribe               job_blue-collar 
                            0                             0 
             job_entrepreneur                 job_housemaid 
                            0                             0 
               job_management                   job_retired 
                            0                             0 
            job_self-employed                  job_services 
                            0                             0 
                  job_student                job_technician 
                            0                             0 
               job_unemployed                   job_unknown 
                            0                             0 
                       job_NA               marital_married 
                            0                             0 
               marital_single               marital_unknown 
                            0                             0 
                   marital_NA            education_basic.6y 
                            0                             0 
           education_basic.9y         education_high.school 
                            0                             0 
         education_illiterate education_professional.course 
                            0                             0 
  education_university.degree             education_unknown 
                            0                             0 
                 education_NA               default_unknown 
                            0                             0 
                  default_yes                    default_NA 
                            0                             0 
              housing_unknown                   housing_yes 
                            0                             0 
                   housing_NA                  loan_unknown 
                            0                             0 
                     loan_yes                       loan_NA 
                            0                             0 
            contact_telephone                    contact_NA 
                            0                             0 
                    month_aug                     month_dec 
                            0                             0 
                    month_jul                     month_jun 
                            0                             0 
                    month_mar                     month_may 
                            0                             0 
                    month_nov                     month_oct 
                            0                             0 
                    month_sep                      month_NA 
                            0                             0 
              day_of_week_mon               day_of_week_thu 
                            0                             0 
              day_of_week_tue               day_of_week_wed 
                            0                             0 
               day_of_week_NA          poutcome_nonexistent 
                            0                             0 
             poutcome_success                   poutcome_NA 
                            0                             0

In [736]:
# Show the train set
head(train)

,client_id,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,⋯,month_sep,month_NA,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,day_of_week_NA,poutcome_nonexistent,poutcome_success,poutcome_NA
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,29925,42,1,999,0,1.4,93.918,-42.7,4.968,5228.1,⋯,0,0,0,1,0,0,0,1,0,0
2,37529,35,4,999,0,1.4,94.465,-41.8,4.960,5228.1,⋯,0,0,1,0,0,0,0,1,0,0
4,9642,45,1,999,0,-1.8,93.075,-47.1,1.453,5099.1,⋯,0,0,0,0,1,0,0,1,0,0
5,14183,45,1,999,0,1.1,93.994,-36.4,4.859,5191.0,⋯,0,0,0,0,0,1,0,1,0,0
6,15180,38,2,999,0,1.1,93.994,-36.4,4.858,5191.0,⋯,0,0,0,0,0,1,0,1,0,0
7,27168,33,1,999,1,-1.8,93.075,-47.1,1.405,5099.1,⋯,0,0,0,0,0,0,0,0,0,0


In [737]:
table(train$subscribe) #get the distribution of the target variable


    0     1 
12130  1583 

## Treating NA values in the test set
following the same steps as for the train

In [738]:
colSums(is.na(test))

client_id            age            job        marital      education 
             0             59             47             52             50 
       default        housing           loan        contact          month 
            54             62             69             65             52 
   day_of_week       campaign          pdays       previous       poutcome 
            54             58             49             68             51 
  emp.var.rate cons.price.idx  cons.conf.idx      euribor3m    nr.employed 
            49             49             46             63             56 
     subscribe 
             0

In [739]:
#replacing some NA containing variables to 0 
cols =  c("emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed")
for (i in cols){
test[is.na(test[,i]), i] <- mean(test[,i], na.rm = TRUE)
}

In [740]:
#replacing pdays with 999 - no contact as most of the values are 999
test$pdays[is.na(test$pdays)] = 999
test$previous[is.na(test$previous)] = 0 # as most of the values are 0

In [741]:
#dropping na observations in campaign and age
test = test[complete.cases(test[ ,"campaign"]),]
test = test[complete.cases(test[ ,"age"]),]

In [742]:
colSums(is.na(test))

client_id            age            job        marital      education 
             0              0             47             51             50 
       default        housing           loan        contact          month 
            54             61             65             65             51 
   day_of_week       campaign          pdays       previous       poutcome 
            53              0              0              0             51 
  emp.var.rate cons.price.idx  cons.conf.idx      euribor3m    nr.employed 
             0              0              0              0              0 
     subscribe 
             0

In [743]:
# Get list of categorical variables in the train set
var_list <- names(test[, 1:(ncol(test)-1)])
cat_list <- var_list[sapply(test[, var_list], class) == 'character']
cat_list

[1] "job"         "marital"     "education"   "default"     "housing"    
 [6] "loan"        "contact"     "month"       "day_of_week" "poutcome"

In [744]:
# Convert categorical variables to dummy variables
dummy_list <- list()
for (v in cat_list) {
    # Create dummy variables
    tmp <- dummy(v, data=test, sep="_", drop=TRUE)
    # Drop the 1st column
    tmp <- tmp[, 2:ncol(tmp), drop=FALSE]
    # Store the results
    dummy_list[[length(dummy_list)+1]] <- tmp
}

# Combine the dummy variables
dummy_df <- do.call(cbind, dummy_list)

# Add the dummy variables to the data frame
test <- cbind(test, dummy_df)

# Drop the original variable
test <- test[, !(names(test) %in% cat_list)]

In [745]:
sum(is.na(test))

[1] 0

In [746]:
dim(test)

[1] 5883   63

In [747]:
# Show the train set
head(test)

,client_id,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,⋯,month_sep,month_NA,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,day_of_week_NA,poutcome_nonexistent,poutcome_success,poutcome_NA
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
3,2757,44,1,999,0,-1.8,92.893,-46.2,1.264,5099.1,⋯,0,0,1,0,0,0,0,1,0,0
8,9097,38,1,999,0,1.1,93.994,-36.4,4.857,5191.0,⋯,0,0,0,0,0,1,0,1,0,0
12,13536,30,1,999,0,1.4,93.918,-42.7,4.963,5228.1,⋯,0,0,0,1,0,0,0,1,0,0
19,7284,35,1,999,0,1.1,93.994,-36.4,4.858,5191.0,⋯,0,0,0,0,0,1,0,1,0,0
23,33523,32,1,999,0,-1.8,92.893,-46.2,1.281,5099.1,⋯,0,0,0,0,0,1,0,1,0,0
24,21812,49,1,999,0,1.1,93.994,-36.4,4.856,5191.0,⋯,0,0,0,0,1,0,0,1,0,0


In [748]:
table(test$subscribe)


   0    1 
5236  647 

## Stepwise feature selection

In [749]:

#http://www.sthda.com/english/articles/37-model-selection-essentials-in-r/154-stepwise-regression-essentials-in-r/
# Set seed for reproducibility
set.seed(123)
# Set up repeated k-fold cross-validation
train.control <- trainControl(method = "cv", number = 10)
# Run a backwards stepwise cross validated feature selection
# where the model with best n variables will be returned
step.model <- train(subscribe ~., data = train,
                    method = "leapBackward", 
                    tuneGrid = data.frame(nvmax = 1:ncol(train)), #from 1 to number of columns in train set
                    trControl = train.control
                    )

# Model accuracy FOR EACH NUMBER OF VARIABLES 
step.model$results

,nvmax,RMSE,Rsquared,MAE,RMSESD,RsquaredSD,MAESD
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0.2995682,0.1224753,0.1796255,0.011051973,0.03399925,0.006534059
2,2,0.2915361,0.1683550,0.1700516,0.009928656,0.03467262,0.005863137
3,3,0.2881495,0.1875697,0.1661000,0.009661797,0.03863934,0.006517532
4,4,0.2877351,0.1899576,0.1653168,0.009467465,0.03578235,0.006331945
5,5,0.2870005,0.1941080,0.1645731,0.010121758,0.03532056,0.006531532
6,6,0.2868101,0.1954040,0.1642506,0.010230601,0.03778650,0.006460860
7,7,0.2865106,0.1973288,0.1639406,0.010303629,0.04003086,0.006543044
8,8,0.2866185,0.1967367,0.1639186,0.010301692,0.04009437,0.006494836
9,9,0.2862604,0.1987662,0.1636004,0.010372887,0.04052264,0.006409886


In [750]:
step.model # results of the backwards selection

Linear Regression with Backwards Selection 

13713 samples
   63 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 12341, 12342, 12342, 12342, 12342, 12342, ... 
Resampling results across tuning parameters:

  nvmax  RMSE       Rsquared   MAE      
   1     0.2995682  0.1224753  0.1796255
   2     0.2915361  0.1683550  0.1700516
   3     0.2881495  0.1875697  0.1661000
   4     0.2877351  0.1899576  0.1653168
   5     0.2870005  0.1941080  0.1645731
   6     0.2868101  0.1954040  0.1642506
   7     0.2865106  0.1973288  0.1639406
   8     0.2866185  0.1967367  0.1639186
   9     0.2862604  0.1987662  0.1636004
  10     0.2861991  0.1991293  0.1635894
  11     0.2860705  0.1998753  0.1634679
  12     0.2861201  0.1995611  0.1634216
  13     0.2861555  0.1993572  0.1633930
  14     0.2861651  0.1992943  0.1634119
  15     0.2861325  0.1995357  0.1633363
  16     0.2860973  0.1997339  0.1633045
  17     0.2860019  0.2002944  0.1632518
  18     0.

In [751]:
step.model$bestTune
#the best model is the one with 21 variables - smallest RMSE

,nvmax
,<int>
21,21


In [752]:
coef(step.model$finalModel, 21) #get coefficients for the model with 21 variables

(Intercept)                    campaign 
              -3.0038815624               -0.0018716150 
                      pdays                emp.var.rate 
              -0.0003631852               -0.0454542016 
             cons.price.idx               cons.conf.idx 
               0.0797242783                0.0063769120 
                nr.employed                 job_retired 
              -0.0007238289                0.0382635876 
                job_student             marital_unknown 
               0.0524539674                0.1592697782 
                 marital_NA education_university.degree 
               0.0549133154                0.0131958190 
            default_unknown                  default_NA 
              -0.0136738399                0.0487171770 
          contact_telephone                   month_dec 
              -0.0334810141                0.1122724980 
                  month_jul                   month_mar 
               0.0342816977                0.2403246277 
                  month_may                   month_nov 
              -0.0657342518               -0.0196961416 
            day_of_week_mon        poutcome_nonexistent 
              -0.0183364785                0.0478246348

In [753]:
summary(step.model$finalModel) #get the summary of the final model. 
#summary shows what variables were selected at each iteration 

Subset selection object
63 Variables  (and intercept)
                              Forced in Forced out
client_id                         FALSE      FALSE
age                               FALSE      FALSE
campaign                          FALSE      FALSE
pdays                             FALSE      FALSE
previous                          FALSE      FALSE
emp.var.rate                      FALSE      FALSE
cons.price.idx                    FALSE      FALSE
cons.conf.idx                     FALSE      FALSE
euribor3m                         FALSE      FALSE
nr.employed                       FALSE      FALSE
`job_blue-collar`                 FALSE      FALSE
job_entrepreneur                  FALSE      FALSE
job_housemaid                     FALSE      FALSE
job_management                    FALSE      FALSE
job_retired                       FALSE      FALSE
`job_self-employed`               FALSE      FALSE
job_services                      FALSE      FALSE
job_student                 

## Applying the model 

In [754]:
#turning target into factor to apply classification models
train$subscribe = as.factor(train$subscribe)
test$subscribe = as.factor(test$subscribe)

In [755]:
levels(test$subscribe) #checking factor variable

[1] "0" "1"

In [756]:
#https://machinelearningmastery.com/machine-learning-in-r-step-by-step/
# Run algorithms using 10-fold cross validation using the variables selected by the stepwise regression
control <- trainControl(method="cv", number=10) #setting up 10 fold cross validation

metric <- "Accuracy" #evaluation metric 

#APPLYING THE ALGORITHMS USING THE VARIABLES SELECTED BY THE STEPWISE SELECTION
# a) linear algorithms

set.seed(7) 
fit.lda <- train(subscribe ~ campaign + pdays + emp.var.rate + cons.price.idx + cons.conf.idx+
        nr.employed+job_retired+job_student+marital_unknown + marital_NA +education_university.degree+
        default_unknown + default_NA + contact_telephone +month_dec+month_jul+month_mar+month_may +
        month_nov + day_of_week_mon +poutcome_nonexistent,
        data=train, method="lda", metric=metric, trControl=control)

# b) nonlinear algorithms
# CART
set.seed(7)
fit.cart <- train(subscribe ~ campaign + pdays + emp.var.rate + cons.price.idx + cons.conf.idx+
        nr.employed+job_retired+job_student+marital_unknown + marital_NA +education_university.degree+
        default_unknown + default_NA + contact_telephone +month_dec+month_jul+month_mar+month_may + 
        month_nov + day_of_week_mon +poutcome_nonexistent,
        data=train, method="rpart", metric=metric, trControl=control)

# kNN
set.seed(7)
fit.knn <- train(subscribe ~ campaign + pdays + emp.var.rate + cons.price.idx + cons.conf.idx+
        nr.employed+job_retired+job_student+marital_unknown + marital_NA +education_university.degree+
        default_unknown + default_NA + contact_telephone +month_dec+month_jul+month_mar+month_may + 
        month_nov + day_of_week_mon +poutcome_nonexistent,
        data=train, method="knn", metric=metric, trControl=control)

# c) advanced algorithms
# SVM
set.seed(7)
fit.svm <- train(subscribe ~ campaign + pdays + emp.var.rate + cons.price.idx + cons.conf.idx+
        nr.employed+job_retired+job_student+marital_unknown + marital_NA +education_university.degree+
        default_unknown + default_NA + contact_telephone +month_dec+month_jul+month_mar+month_may + 
        month_nov + day_of_week_mon +poutcome_nonexistent,
        data=train, method="svmRadial", metric=metric, trControl=control)
# Random Forest
set.seed(7)
fit.rf <- train(subscribe ~ campaign + pdays + emp.var.rate + cons.price.idx + cons.conf.idx+
        nr.employed+job_retired+job_student+marital_unknown + marital_NA +education_university.degree+
        default_unknown + default_NA + contact_telephone +month_dec+month_jul+month_mar+month_may + 
        month_nov + day_of_week_mon +poutcome_nonexistent,
        data=train, method="rf", metric=metric, trControl=control)


In [758]:
# summarize accuracy of models
results <- resamples(list(lda=fit.lda, cart=fit.cart, knn=fit.knn, svm=fit.svm, rf=fit.rf), 
                     measures=list(acc, mlr::auc))
summary(results)


Call:
summary.resamples(object = results)

Models: lda, cart, knn, svm, rf 
Number of resamples: 10 

Accuracy 
          Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lda  0.8724490 0.8876937 0.8898614 0.8885007 0.8918673 0.8957726    0
cart 0.8826531 0.8936907 0.8967907 0.8950639 0.8982495 0.9008746    0
knn  0.8884840 0.8936907 0.8942378 0.8952089 0.8957536 0.9022611    0
svm  0.8927790 0.8937490 0.8964260 0.8964489 0.8986142 0.9022611    0
rf   0.8855685 0.8944779 0.8982495 0.8963764 0.8991612 0.9008023    0

Kappa 
          Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lda  0.2902833 0.3570532 0.3735037 0.3693521 0.3902179 0.4176249    0
cart 0.2090662 0.2622400 0.2951138 0.2878451 0.3263884 0.3556263    0
knn  0.2309530 0.2851491 0.3072071 0.3077430 0.3420934 0.3514797    0
svm  0.2093520 0.2374064 0.2655732 0.2613638 0.2831645 0.3015686    0
rf   0.1588804 0.2388747 0.2723059 0.2567604 0.2893886 0.2929814    0


In [759]:
# summarize Best Model 
print(fit.svm)

Support Vector Machines with Radial Basis Function Kernel 

13713 samples
   21 predictor
    2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 12342, 12342, 12342, 12342, 12341, 12342, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.8964489  0.2613638
  0.50  0.8961573  0.2686502
  1.00  0.8951365  0.2779010

Tuning parameter 'sigma' was held constant at a value of 0.04781915
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.04781915 and C = 0.25.


## Making predictions

In [760]:
# estimate skill of svm on the validation dataset
predictions <- predict(fit.svm, test)
confusionMatrix(predictions, test$subscribe)

Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 5178  515
         1   58  132
                                          
               Accuracy : 0.9026          
                 95% CI : (0.8947, 0.9101)
    No Information Rate : 0.89            
    P-Value [Acc > NIR] : 0.0009327       
                                          
                  Kappa : 0.2794          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.9889          
            Specificity : 0.2040          
         Pos Pred Value : 0.9095          
         Neg Pred Value : 0.6947          
             Prevalence : 0.8900          
         Detection Rate : 0.8802          
   Detection Prevalence : 0.9677          
      Balanced Accuracy : 0.5965          
                                          
       'Positive' Class : 0               
                        

In [792]:
# AUC
pred_roc <- pROC::roc(as.numeric(predictions), as.numeric(test$subscribe == "1"))
pred_auc <- pROC::auc(pred_roc)
print(paste('AUC =', pred_auc))

Setting levels: control = 1, case = 2

Setting direction: controls < cases



[1] "AUC = 0.802137435631939"


In [793]:
write.csv(train,".//Dataset 1_Bank Marketing//train.csv", row.names = TRUE)
write.csv(test,".//Dataset 1_Bank Marketing//test.csv", row.names = TRUE)